In [76]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from IPython.display import display
from ast import literal_eval
import os

mpl.rc("font", family="serif", size=15)

In [118]:
def find_nearest(trgeff, target):
    trgeff = np.asarray(trgeff)
    idx = (np.abs(trgeff - target)).argmin()
    return int(trgeff[idx])

def fom(S,B):
    return np.sqrt(2 * ( (S+B) * np.log(1+S/B) - S) )

def divisorGenerator(n):
    large_divisors = []
    for i in range(1, int(np.sqrt(n) + 1)):
        if n % i == 0:
            yield i
            if i*i != n:
                large_divisors.append(n / i)
    for divisor in reversed(large_divisors):
        yield divisor

def rebinCalc(nBins,target):
    rebinFloat = nBins/float(target)
    allDivs = list(divisorGenerator(nBins))
    return find_nearest(allDivs, rebinFloat)

def colorDict(rocData):
    colorDict = {}
    sigLabels = np.unique(rocData["sig"])
    bkgLabels = np.unique(rocData["bkg"])
    for sigl in sigLabels:
        sigColor = np.unique(rocData[rocData["sig"] == sigl]["cSig"])[0]
        colorDict[sigl] = sigColor
    for bkgl in bkgLabels:
        bkgColor = np.unique(rocData[rocData["bkg"] == bkgl]["cBg"])[0]
        colorDict[bkgl] = bkgColor
    return colorDict

def showPotentialVar(inputFolder,data,cut):
    sigLabels = np.unique(data["sig"])
    bkgLabels = np.unique(data["bkg"])
    display(data.head(20))
    ROCBkg = {}
    for bkgl in bkgLabels:
        ROCBkg[bkgl] = []

    for sigl in sigLabels:
        bkgDFList = []
        for bkgl in bkgLabels:
            oneSignal = data[data["sig"]==sigl]
            oneBkgforOneSig = oneSignal[oneSignal["bkg"] == bkgl]
            obosROC = oneBkgforOneSig.sort_values("roc_auc",ascending=False)[["bkg","var","roc_auc"]]
            obosROC = obosROC.set_index(pd.Index(list(range(len(obosROC)))))
            bkgDFList.append(obosROC)
            # only consider ROC > 0.65
            ROCBkg[bkgl] += list(obosROC[obosROC["roc_auc"] > 0.65]["var"])

    jetVarDict = [
                    "Tau",
                    "Axismajor",
                    "Axisminor",
                    "Pt",
                    "Girth",
                    "SoftDropMass"
                ]
            
    rePDList = []
    plotWorthy = []
    newROCCut = 0
    for bkgl in bkgLabels:
        ROCBkg[bkgl]
        unique, counts = np.unique(ROCBkg[bkgl], return_counts=True)
        bkg = [bkgl]*len(unique)
        result = np.column_stack((bkg,unique,counts))
        rePD = pd.DataFrame(result,columns=["bkg","var","counts"]).sort_values("counts",ascending=False)
        rePD["counts"] = pd.to_numeric(rePD["counts"])
        rePD = rePD.set_index(pd.Index(list(range(len(rePD)))))
        rePDList.append(rePD)
        useful = rePD[rePD["counts"] >= 5]
        if len(useful.index) > newROCCut:
            newROCCut = len(useful.index)
        plotWorthy += list(useful["var"])
    newROC = pd.concat(rePDList,axis=1)
    varsOfInterest = []
    for var in np.unique(plotWorthy):
        jetVar = False
        for key in jetVarDict:
            if key in var:
                jetVar = True
                break
        if jetVar == False:
            varsOfInterest.append(var)
    display(newROC.head(newROCCut))
#     newROC.head(newROCCut).to_csv("{}/{}/usefulVar.csv".format(inputFolder,cut))
    newROC.to_csv("{}/{}/usefulVar.csv".format(inputFolder,cut))
    return varsOfInterest

def nearestCut(sigEff, cutVal, target):
    sigEff = np.asarray(sigEff)
    idx = (np.abs(sigEff - target)).argmin()
    return float(cutVal[idx])

def completeCutYields(varOfInterest,rocData,yieldData,inputFolder,cut):
    comCYTable = pd.DataFrame(columns=["var","cutValues","cutDir","sigLabel","sgCutYield","bkgLabel","bgCutYield"])
    cutYieldData = yieldData[(yieldData["var"]=="njetsAK8") & (yieldData["cut"]==cut) ]

    sigLabels = np.unique(rocData["sig"])
    for var in varOfInterest:
        unique, counts = np.unique(rocData[rocData["var"] == var]["cutDir"], return_counts=True)
        mostCutDir = unique[np.argmax(counts)]
        for sigl in sigLabels:
            for bkgl in ["QCD","TTJets (pow)","W+jets","Z+jets"]:
                condition = (rocData["sig"] == sigl) & (rocData["bkg"] == bkgl) & (rocData["var"] == var)
                rowROCData = rocData[condition].iloc[0]
                if rowROCData["cutDir"] == mostCutDir:
                    bgCutYield = np.array(rowROCData["mBg_f"])
                    sgCutYield = np.array(rowROCData["mSig_f"])
                else:
                    bgCutYield = 1 - np.array(rowROCData["mBg_f"])
                    sgCutYield = 1 - np.array(rowROCData["mSig_f"])
                bgCutYield *= cutYieldData[cutYieldData["source"] == bkgl]["yield"].iloc[0]
                sgCutYield *= cutYieldData[cutYieldData["source"] == sigl]["yield"].iloc[0]
                comCYTable.loc[len(comCYTable.index)] = [var,rowROCData["cutSig"],rowROCData["cutDir"],sigl,sgCutYield,bkgl,bgCutYield]
    display(comCYTable)
    comCYTable.to_pickle("{}/{}/comCYTable.pkl".format(inputFolder,cut))

def cutSug(bkgYields,sigYields,cutDir,cutValues):
    foms = fom(sigYields,bkgYields)
    foms = np.nan_to_num(foms,nan=0)
    foms = np.where(foms>10000,0,foms)
    fomforCut = "{},{:.3f}".format(cutDir,np.amax(foms))
    suggestedCut = cutValues[np.argmax(foms)]
    return [fomforCut,suggestedCut]
    
# using the comCYTable, make suggested cuts based on the maximum FOM. Function should also have the 
# ability to choose which background to use for calculating the FOM.
def maxFOMSuggest(inputFolder,cut,pltColorDict):
    comCYTable = pd.read_pickle("{}/{}/comCYTable.pkl".format(inputFolder,cut))
    sigLabels = np.unique(comCYTable["sigLabel"])
    varOfInterest = np.unique(comCYTable["var"])
    sugCuts = pd.DataFrame(columns=["var","sig","QCD FOM","QCD Cut","TT FOM","TT Cut","WJet FOM","WJet Cut","ZJet FOM","ZJet Cut","W+ & Z+Jets FOM","W+ & Z+Jets Cut","All FOM","All Cut"])
    for var in varOfInterest:
        unique, counts = np.unique(comCYTable[comCYTable["var"] == var]["cutDir"], return_counts=True)
        mostCutDir = unique[np.argmax(counts)].replace("cut","")
        for sigl in sigLabels:
            sigCond = (comCYTable["sigLabel"] == sigl) & (comCYTable["var"] == var) & (comCYTable["bkgLabel"] == "QCD")
            cySigData = comCYTable[sigCond].iloc[0]
            cutValues = cySigData["cutValues"]
            sigYields = cySigData["sgCutYield"]
            allBkgYields = np.zeros(len(sigYields))
            ewBkgYields = np.zeros(len(sigYields))
            maxFOMs = [sigl]
            for bkgl in ["QCD","TTJets (pow)","W+jets","Z+jets"]:
                bkgCond = (comCYTable["bkgLabel"] == bkgl) & (comCYTable["var"] == var) & (comCYTable["sigLabel"] == "baseline")
                cyBkgData = comCYTable[bkgCond].iloc[0]
                bkgYields = cyBkgData["bgCutYield"]
                allBkgYields += bkgYields
                if bkgl in ["W+jets","Z+jets"]:
                    ewBkgYields += bkgYields
                maxFOMs += cutSug(bkgYields,sigYields,mostCutDir,cutValues)
            maxFOMs += cutSug(ewBkgYields,sigYields,mostCutDir,cutValues)  
            maxFOMs += cutSug(allBkgYields,sigYields,mostCutDir,cutValues)            
            sugCuts.loc[len(sugCuts.index)] = [var] + maxFOMs
        QCDAvgCut = np.mean(np.array(sugCuts[sugCuts["var"] == var]["QCD Cut"]))
        TTAvgCut = np.mean(np.array(sugCuts[sugCuts["var"] == var]["TT Cut"]))
        WJetAvgCut = np.mean(np.array(sugCuts[sugCuts["var"] == var]["WJet Cut"]))
        ZJetAvgCut = np.mean(np.array(sugCuts[sugCuts["var"] == var]["ZJet Cut"]))
        ewAvgCut = np.mean(np.array(sugCuts[sugCuts["var"] == var]["W+ & Z+Jets Cut"]))
        allAvgCut = np.mean(np.array(sugCuts[sugCuts["var"] == var]["All Cut"]))
        sugCuts.loc[len(sugCuts.index)] = [var,"Average","",QCDAvgCut,"",TTAvgCut,"",WJetAvgCut,"",ZJetAvgCut,"",ewAvgCut,"",allAvgCut]
    sugCuts = sugCuts[["var","sig","QCD FOM","TT FOM","WJet FOM","ZJet FOM","W+ & Z+Jets FOM","All FOM","QCD Cut","TT Cut","WJet Cut","ZJet Cut","W+ & Z+Jets Cut","All Cut"]]
    # calculate the average cut values
    sugCuts.to_csv("{}/{}/suggestedCutsByFOM.csv".format(inputFolder,cut),index=False)
    for var in varOfInterest:
        figure, axis = plt.subplots(2, 2, figsize=(12,12))
        pos = [axis[0,0],axis[0,1],axis[1,0],axis[1,1]]
        figure.suptitle(var, fontsize=16)
        allBkgYields = []
        ewBkgYields = []
        QCDYields = [] 
        TTYields = []
        for bkgl in ["QCD","TTJets (pow)","W+jets","Z+jets"]:
            bkgCond = (comCYTable["bkgLabel"] == bkgl) & (comCYTable["var"] == var) & (comCYTable["sigLabel"] == "baseline")
            cyBkgData = comCYTable[bkgCond].iloc[0]
            bkgYields = cyBkgData["bgCutYield"]
            if allBkgYields == []:
                allBkgYields = bkgYields
            else:
                allBkgYields += bkgYields
            if (ewBkgYields == []) and (bkgl in ["W+jets","Z+jets"]):
                ewBkgYields = bkgYields
            elif (ewBkgYields != []) and (bkgl in ["W+jets","Z+jets"]):
                ewBkgYields += bkgYields
            if bkgl == "QCD":
                QCDYields = bkgYields
            elif bkgl == "TTJets (pow)":
                TTYields = bkgYields
        bkgYieldDict = {"QCD":QCDYields,"TT":TTYields,"W+ & Z+Jets":ewBkgYields,"All":allBkgYields}
        for sigl in sigLabels:
            sigCond = (comCYTable["sigLabel"] == sigl) & (comCYTable["var"] == var) & (comCYTable["bkgLabel"] == "QCD")
            cySigData = comCYTable[sigCond].iloc[0]
            cutValues = cySigData["cutValues"]
            sigYields = cySigData["sgCutYield"]
            i = 0
            for bkgl,bkgYields in bkgYieldDict.items():
                indFOM = fom(sigYields,bkgYields)
                indFOM = np.nan_to_num(indFOM,nan=0)
                indFOM = np.where(indFOM>10000,0,indFOM)
                pos[i].plot(cutValues,indFOM,color=pltColorDict[sigl],linewidth=2,label=sigl)
                pos[i].set_yscale("log")
                pos[i].legend(fontsize=11)
                cond = (sugCuts["sig"] == "Average") & (sugCuts["var"] == var)
                avgCut = sugCuts[cond].iloc[0]
                pos[i].vlines(avgCut["{} Cut".format(bkgl)],0.001,200)
                pos[i].set_title(bkgl)
                pos[i].set_xticks(np.round_(np.linspace(min(cutValues),max(cutValues),20),2))
                pos[i].set_ylim(0.001,200)
                plt.setp(pos[i].xaxis.get_majorticklabels(), rotation=90)
                pos[i].grid(b=True)
                i += 1
        plt.savefig("{}/{}/indFOM_{}.png".format(inputFolder,cut,var))
        plt.close()
            
            
def plotVarOfInterest(varOfInterest,rocData,inputFolder,cut="",sigEffTarget=0.8):
    varOfInterest += ["dPhij23","dPhij23AK8","dPhiMinjMETAK8"]
    varOfInterest = np.unique(varOfInterest)
    sigLabels = np.unique(rocData["sig"])
    bkgLabels = np.unique(rocData["bkg"])
    if not os.path.exists("{}/{}".format(inputFolder,cut)):
        os.makedirs("{}/{}".format(inputFolder,cut))
    sugCutsSigEffTar = []
    for var in varOfInterest:
        figure, axis = plt.subplots(2, 2, figsize=(12,12))
        pos = [axis[0,0],axis[0,1],axis[1,0],axis[1,1]]
        figure.suptitle(var, fontsize=16)
        cutThs = []
        for i in range(len(bkgLabels)):
            bkgl = bkgLabels[i]
            for sigl in sigLabels:
                condition = (rocData["sig"] == sigl) & (rocData["bkg"] == bkgl) & (rocData["var"] == var)
                rowROCData = rocData[condition].iloc[0]
                xval = rowROCData["mBg_f"]
                yval = rowROCData["mSig_f"]
                cutval = rowROCData["cutSig"]
                rocScore = rowROCData["roc_auc"]
                rebin = rebinCalc(len(cutval),20)
                color = rowROCData["cSig"]
                cutDir = rowROCData["cutDir"]
                cutThs.append([sigl,cutDir,nearestCut(yval, cutval, sigEffTarget)])
                for j in range(len(cutval)):
                    if j%rebin == 0:
                        pos[i].text(xval[j],yval[j],"{:.2f}".format(cutval[j]),fontsize=10,color="black")
                pos[i].plot(xval,yval,color=color,linewidth=2,marker=".",label="{} ({},{})".format(sigl.replace("M-2000_",""),rocScore,cutDir.replace("cut","")))
            pos[i].plot([0,1],[0,1],linestyle=":",linewidth=2,color="black")
            pos[i].set_title(bkgl)
            pos[i].legend(fontsize=11)
            pos[i].set_ylabel(r"$\epsilon_{sig}$")
            pos[i].set_xlabel(r"$\epsilon_{bkg}$")
        cutThs = np.array(cutThs)
        # figure out the most common cut direction
        unique, counts = np.unique(cutThs[:,1], return_counts=True)
        mostCutDir = unique[np.argmax(counts)]
        # we only consider entries with the most common cut direction
        cutThs = cutThs[cutThs[:,1]==mostCutDir]
        cutThs = np.unique(cutThs,axis=0)
        # we want the loosest cut across signals, so that the signal with the worst efficiency
        # suffers as little as possible from the cut
        if mostCutDir == ">=cut":
            bestCut = np.amin(cutThs[:,2].astype(float))
        else:
            bestCut = np.amax(cutThs[:,2].astype(float))
        sugCutsSigEffTar.append("{} {} {}".format(var,mostCutDir.replace("cut",""),bestCut))
        plt.xlim(0,1)
        plt.ylim(0,1)
        plt.savefig("{}/{}/ROC_{}.png".format(inputFolder,cut,var))
        plt.close()
    with open('{}/{}/suggestedCutsBySigEff.txt'.format(inputFolder,cut), 'w') as f:
        f.write('\n'.join(sugCutsSigEffTar))
        
def bkgCheck(slabel):
    bkgLabels = ["QCD","TTJets (pow)","W+jets","Z+jets"]
    isbkg = False
    for bl in bkgLabels:
        if slabel == bl:
            isbkg = True
            break
    return isbkg

def FOMTable(yieldData,inputFolder,cut):
    cutYieldData = yieldData[(yieldData["var"]=="njetsAK8") & (yieldData["cut"]==cut) ]
    display(cutYieldData)
    
    QCDYield = cutYieldData[cutYieldData["source"] == "QCD"]["yield"].iloc[0]
    TTYield = cutYieldData[cutYieldData["source"] == "TTJets (pow)"]["yield"].iloc[0]
    WYield = cutYieldData[cutYieldData["source"] == "W+jets"]["yield"].iloc[0]
    ZYield = cutYieldData[cutYieldData["source"] == "Z+jets"]["yield"].iloc[0]
    bkgYields = [QCDYield,TTYield,WYield,ZYield,np.sum([QCDYield,TTYield,WYield,ZYield])]

    fomTable = pd.DataFrame(columns=["sig","QCD","TT","W+Jets","Z+Jets","All"])
    for slabel in cutYieldData["source"]:
        if not bkgCheck(slabel):
            fomList = [slabel]
            sigYield = cutYieldData[cutYieldData["source"] == slabel]["yield"].iloc[0]
            fomList += list(fom(sigYield,bkgYields))
            fomTable.loc[len(fomTable.index)] = fomList
    print(cut)
    display(fomTable)
    fomTable.to_csv("{}/{}/fomTable.csv".format(inputFolder,cut),index=False)

def plotFOMDiff(inputFolder,oriCut,newCut):
    fomOld = pd.read_csv("{}/{}/fomTable.csv".format(inputFolder,oriCut),index_col="sig")
    fomNew = pd.read_csv("{}/{}/fomTable.csv".format(inputFolder,newCut),index_col="sig")
    newvsold = fomNew.div(fomOld)
    # getting color dictionary for signals
    data = pd.read_pickle("{}/allRocValues.pkl".format(inputFolder))
    data = data[(data["cut"]==oriCut) & (data["var"]=="njetsAK8")]
    sigLabels = np.unique(data["sig"])
    sigColDict = {}
    for sl in sigLabels:
        sigColDict[sl] = data[data["sig"]==sl]["cSig"].iloc[0]
    print(newCut)
    display(newvsold)
    newvsold.to_csv("{}/{}/fomNewVsOld.csv".format(inputFolder,newCut),index=False)
    plt.figure(figsize=(12,8))
    for sl in sigLabels:
        plt.plot(list(newvsold.columns),list(newvsold.loc[[sl]].iloc[0]),color=sigColDict[sl],label=sl)
    plt.legend()
    plt.title(newCut)
    plt.hlines(1.0,0,len(list(newvsold.columns)))
    plt.xlabel("Background")
    plt.ylabel("FOM After/FOM Before Cut")
    plt.savefig("{}/{}/fomNewVsOld.png".format(inputFolder,newCut))
    plt.close()
    
def performStudy(inputFolder,oriCut):
    rocData = pd.read_pickle("{}/allRocValues.pkl".format(inputFolder))
    rocData = rocData.replace("Z#rightarrow#nu#nu+jets","Z+jets")
    rocData = rocData.replace("t#bar{t} (pow)","TTJets (pow)")
    cuts = np.unique(rocData["cut"])
    yieldData = pd.read_pickle("{}/yieldValues.pkl".format(inputFolder))
    yieldData = yieldData.replace("Z#rightarrow#nu#nu+jets","Z+jets")
    yieldData = yieldData.replace("t#bar{t}","TTJets (pow)")
    pltColorDict = colorDict(rocData)
    for cut in cuts:
        if not os.path.exists("{}/{}".format(inputFolder,cut)):
            os.makedirs("{}/{}".format(inputFolder,cut))
        cutROCData = rocData[rocData["cut"] == cut]
        varsOfInterest = showPotentialVar(inputFolder,cutROCData,cut)
        completeCutYields(varsOfInterest,cutROCData,yieldData,inputFolder,cut)
        maxFOMSuggest(inputFolder,cut,pltColorDict)
        plotVarOfInterest(varsOfInterest,cutROCData,inputFolder,cut=cut)
        FOMTable(yieldData,inputFolder,cut)
    for cut in cuts:
        plotFOMDiff(inputFolder,oriCut,cut)

In [119]:
performStudy("UL_jetalt2p4_trigger_qual_FlorianCuts_2018_morejN_count","_qual_trg")

/home/keane/anaconda3/lib/python3.7/site-packages/pandas/core/missing.py:94: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= arr == x


,cut,var,sig,bkg,roc_auc,cutDir,cutSig,cBg,cSig,mBg_f,mSig_f
0,_qual_trg,j1PtD,mMed 600,Z+jets,0.55,<=cut,"[0.0, 0.03, 0.06, 0.09, 0.12, 0.15, 0.18, 0.21...",#999999,#cd2bcc,"[-2.220446049250313e-16, 4.081572031688907e-06...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0007125679166..."
1,_qual_trg,j1PtD,baseline,Z+jets,0.64,<=cut,"[0.0, 0.03, 0.06, 0.09, 0.12, 0.15, 0.18, 0.21...",#999999,#cc660d,"[-2.220446049250313e-16, 4.081572031688907e-06...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0026780279975..."
2,_qual_trg,j1PtD,M-2000_mD-1,Z+jets,0.53,>=cut,"[0.0, 0.03, 0.06, 0.09, 0.12, 0.15, 0.18, 0.21...",#999999,#f2231b,"[1.0000000000000002, 0.9999959184279683, 0.999...","[1.0000000000000002, 1.0000000000000002, 1.000..."
3,_qual_trg,j1PtD,M-2000_mD-100,Z+jets,0.55,<=cut,"[0.0, 0.03, 0.06, 0.09, 0.12, 0.15, 0.18, 0.21...",#999999,#9a27cc,"[-2.220446049250313e-16, 4.081572031688907e-06...","[-2.220446049250313e-16, -2.220446049250313e-1..."
4,_qual_trg,j1PtD,M-2000_r-0p1,Z+jets,0.66,<=cut,"[0.0, 0.03, 0.06, 0.09, 0.12, 0.15, 0.18, 0.21...",#999999,#663303,"[-2.220446049250313e-16, 4.081572031688907e-06...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.000272108843537233..."
5,_qual_trg,j1PtD,M-2000_r-0p7,Z+jets,0.57,<=cut,"[0.0, 0.03, 0.06, 0.09, 0.12, 0.15, 0.18, 0.21...",#999999,#f69acc,"[-2.220446049250313e-16, 4.081572031688907e-06...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0002383506137..."
6,_qual_trg,j1PtD,M-2000_a-low,Z+jets,0.63,<=cut,"[0.0, 0.03, 0.06, 0.09, 0.12, 0.15, 0.18, 0.21...",#999999,#cccccc,"[-2.220446049250313e-16, 4.081572031688907e-06...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0006263048016..."
7,_qual_trg,j1PtD,M-2000_a-high,Z+jets,0.59,<=cut,"[0.0, 0.03, 0.06, 0.09, 0.12, 0.15, 0.18, 0.21...",#999999,#999910,"[-2.220446049250313e-16, 4.081572031688907e-06...","[1.1102230246251565e-16, 1.1102230246251565e-1..."
8,_qual_trg,j1PtD,mMed 600,W+jets,0.56,<=cut,"[0.0, 0.03, 0.06, 0.09, 0.12, 0.15, 0.18, 0.21...",#cccc19,#cd2bcc,"[0.0, 1.4628646354597485e-05, 1.46286463545974...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0007125679166..."
9,_qual_trg,j1PtD,baseline,W+jets,0.64,<=cut,"[0.0, 0.03, 0.06, 0.09, 0.12, 0.15, 0.18, 0.21...",#cccc19,#cc660d,"[0.0, 1.4628646354597485e-05, 1.46286463545974...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0026780279975..."


,bkg,var,counts,bkg,var,counts,bkg,var,counts,bkg,var,counts
0,QCD,dRj12AK8,8,TTJets (pow),j2Tau21AK8,7.0,W+jets,st,8.0,Z+jets,METrHT_pt30,8
1,QCD,met,8,TTJets (pow),j1Tau21AK8,7.0,W+jets,j1PtAK8,8.0,Z+jets,ht,8
2,QCD,dRj13AK8,8,TTJets (pow),st,6.0,W+jets,dPhiMinjMET,8.0,Z+jets,njetsAK8,8
3,QCD,dRj12,8,TTJets (pow),j1Pt,6.0,W+jets,njetsAK8,8.0,Z+jets,njets,8
4,QCD,METrST_pt30,7,TTJets (pow),met,6.0,W+jets,njets,8.0,Z+jets,mT,8
5,QCD,dPhij23AK8,7,TTJets (pow),j1AxismajorAK8,5.0,W+jets,ht,8.0,Z+jets,j3Pt,8
6,QCD,METrHT_pt30,7,TTJets (pow),j1GirthAK8,5.0,W+jets,j2Pt,8.0,Z+jets,METrST_pt30,8
7,QCD,nb,7,TTJets (pow),j2Pt,5.0,W+jets,j1Pt,8.0,Z+jets,j2Pt,8
8,QCD,dPhij12AK8,6,TTJets (pow),j1PtAK8,4.0,W+jets,nb,7.0,Z+jets,j1PtAK8,8
9,QCD,dEtaj23AK8,6,TTJets (pow),j3PtAK8,4.0,W+jets,mT,7.0,Z+jets,j1Pt,8


,var,cutValues,cutDir,sigLabel,sgCutYield,bkgLabel,bgCutYield
0,METrHT_pt30,"[0.0, 0.03, 0.06, 0.09, 0.12, 0.15, 0.18, 0.21...",>=cut,M-2000_a-high,"[-4.978240042419202e-13, 50.58884297520605, 14...",QCD,"[-3.462609132753869e-08, 38186824.73531552, 89..."
1,METrHT_pt30,"[0.0, 0.03, 0.06, 0.09, 0.12, 0.15, 0.18, 0.21...",>=cut,M-2000_a-high,"[-4.978240042419202e-13, 50.58884297520605, 14...",TTJets (pow),"[-8.361384917776604e-10, 119308.32628942636, 2..."
2,METrHT_pt30,"[0.0, 0.03, 0.06, 0.09, 0.12, 0.15, 0.18, 0.21...",<=cut,M-2000_a-high,"[-4.978240042419202e-13, 50.58884297520605, 14...",W+jets,"[-2.688643085946296e-09, 14529.0685849826, 458..."
3,METrHT_pt30,"[0.0, 0.03, 0.06, 0.09, 0.12, 0.15, 0.18, 0.21...",<=cut,M-2000_a-high,"[-4.978240042419202e-13, 50.58884297520605, 14...",Z+jets,"[3.101046086584347e-10, 3066.9782375588484, 10..."
4,METrHT_pt30,"[0.0, 0.03, 0.06, 0.09, 0.12, 0.15, 0.18, 0.21...",>=cut,M-2000_a-low,"[-2.575717417130363e-13, 71.6124306790829, 171...",QCD,"[-3.462609132753869e-08, 38186824.73531552, 89..."
...,...,...,...,...,...,...,...
635,st,"[0.0, 10.0, 20.0, 30.0, 40.0, 50.0, 60.0, 70.0...",>=cut,baseline,"[1001.9999999999987, 1001.9999999999987, 1001....",Z+jets,"[2793174.000000002, 2793173.2785312394, 279316..."
636,st,"[0.0, 10.0, 20.0, 30.0, 40.0, 50.0, 60.0, 70.0...",>=cut,mMed 600,"[171527.00000000015, 171527.00000000015, 17152...",QCD,"[155942052.00000003, 155932347.0052321, 155910..."
637,st,"[0.0, 10.0, 20.0, 30.0, 40.0, 50.0, 60.0, 70.0...",>=cut,mMed 600,"[171527.00000000015, 171527.00000000015, 17152...",TTJets (pow),"[1255210.9999999984, 1255210.802071027, 125521..."
638,st,"[0.0, 10.0, 20.0, 30.0, 40.0, 50.0, 60.0, 70.0...",>=cut,mMed 600,"[171527.00000000015, 171527.00000000015, 17152...",W+jets,"[3027143.0000000056, 3027105.283861941, 302699..."


/home/keane/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in sqrt
  import sys
/home/keane/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys
/home/keane/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in log
  import sys
/home/keane/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys
/home/keane/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:183: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/home/keane/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:187: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/home/keane/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:189: DeprecationWarning: e

...

,sig,QCD,TT,W+Jets,Z+Jets,All
0,mMed 600,6.559232,74.032488,115.376211,143.290138,6.521340
1,baseline,0.103955,1.214120,1.987451,2.582547,0.103354
2,M-2000_mD-1,0.209923,2.450275,4.006925,5.200983,0.208710
3,M-2000_mD-100,0.099425,1.161234,1.900962,2.470279,0.098850
4,M-2000_r-0p1,0.100431,1.172987,1.920183,2.495229,0.099851
5,M-2000_r-0p7,0.081050,0.946726,1.550081,2.014701,0.080581
6,M-2000_a-low,0.117044,1.366885,2.237239,2.906731,0.116367
7,M-2000_a-high,0.110248,1.287568,2.107554,2.738432,0.109611


_qual_trg


,QCD,TT,W+Jets,Z+Jets,All
sig,,,,,
mMed 600,1.0,1.0,1.0,1.0,1.0
baseline,1.0,1.0,1.0,1.0,1.0
M-2000_mD-1,1.0,1.0,1.0,1.0,1.0
M-2000_mD-100,1.0,1.0,1.0,1.0,1.0
M-2000_r-0p1,1.0,1.0,1.0,1.0,1.0
M-2000_r-0p7,1.0,1.0,1.0,1.0,1.0
M-2000_a-low,1.0,1.0,1.0,1.0,1.0
M-2000_a-high,1.0,1.0,1.0,1.0,1.0


_qual_trg_st


,QCD,TT,W+Jets,Z+Jets,All
sig,,,,,
mMed 600,0.477619,0.494217,1.181208,1.410229,0.485509
baseline,1.295568,1.357719,3.451192,4.307799,1.316978
M-2000_mD-1,1.844795,1.932240,4.906453,6.117545,1.875282
M-2000_mD-100,1.051297,1.101784,2.800711,3.496034,1.068669
M-2000_r-0p1,1.729865,1.812823,4.608276,5.752283,1.758447
M-2000_r-0p7,0.941511,0.986816,2.508927,3.132412,0.957072
M-2000_a-low,1.260009,1.320381,3.355820,4.188185,1.280830
M-2000_a-high,1.228134,1.287026,3.271270,4.082953,1.248432


# Round 2

In [4]:
performStudy("preStudy_run2","_qual_trg")

/home/keane/anaconda3/lib/python3.7/site-packages/pandas/core/missing.py:94: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= arr == x


,cut,var,sig,bkg,roc_auc,cutDir,cutSig,cBg,cSig,mBg_f,mSig_f
0,_qual_trg,j1PtD,mMed 600,Z+jets,0.55,<=cut,"[0.0, 0.03, 0.06, 0.09, 0.12, 0.15, 0.18, 0.21...",#999999,#cd2bcc,"[-2.220446049250313e-16, 4.081572031688907e-06...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0007125679166..."
1,_qual_trg,j1PtD,baseline,Z+jets,0.64,<=cut,"[0.0, 0.03, 0.06, 0.09, 0.12, 0.15, 0.18, 0.21...",#999999,#cc660d,"[-2.220446049250313e-16, 4.081572031688907e-06...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0026780279975..."
2,_qual_trg,j1PtD,M-2000_mD-1,Z+jets,0.53,>=cut,"[0.0, 0.03, 0.06, 0.09, 0.12, 0.15, 0.18, 0.21...",#999999,#f2231b,"[1.0000000000000002, 0.9999959184279683, 0.999...","[1.0000000000000002, 1.0000000000000002, 1.000..."
3,_qual_trg,j1PtD,M-2000_mD-100,Z+jets,0.55,<=cut,"[0.0, 0.03, 0.06, 0.09, 0.12, 0.15, 0.18, 0.21...",#999999,#9a27cc,"[-2.220446049250313e-16, 4.081572031688907e-06...","[-2.220446049250313e-16, -2.220446049250313e-1..."
4,_qual_trg,j1PtD,M-2000_r-0p1,Z+jets,0.66,<=cut,"[0.0, 0.03, 0.06, 0.09, 0.12, 0.15, 0.18, 0.21...",#999999,#663303,"[-2.220446049250313e-16, 4.081572031688907e-06...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.000272108843537233..."


,bkg,var,counts,bkg,var,counts,bkg,var,counts,bkg,var,counts
0,QCD,dRj12AK8,8.0,TTJets (pow),j2Tau21AK8,7.0,W+jets,st,8.0,Z+jets,METrHT_pt30,8
1,QCD,met,8.0,TTJets (pow),j1Tau21AK8,7.0,W+jets,j1PtAK8,8.0,Z+jets,ht,8
2,QCD,dRj13AK8,8.0,TTJets (pow),st,6.0,W+jets,dPhiMinjMET,8.0,Z+jets,njetsAK8,8
3,QCD,dRj12,8.0,TTJets (pow),j1Pt,6.0,W+jets,njetsAK8,8.0,Z+jets,njets,8
4,QCD,METrST_pt30,7.0,TTJets (pow),met,6.0,W+jets,njets,8.0,Z+jets,mT,8
5,QCD,dPhij23AK8,7.0,TTJets (pow),j1AxismajorAK8,5.0,W+jets,ht,8.0,Z+jets,j3Pt,8
6,QCD,METrHT_pt30,7.0,TTJets (pow),j1GirthAK8,5.0,W+jets,j2Pt,8.0,Z+jets,METrST_pt30,8
7,QCD,nb,7.0,TTJets (pow),j2Pt,5.0,W+jets,j1SoftDropMassAK8,8.0,Z+jets,j1SoftDropMassAK8,8
8,QCD,dEtaj23AK8,6.0,TTJets (pow),j1PtAK8,4.0,W+jets,j1Pt,8.0,Z+jets,j1PtAK8,8
9,QCD,dPhij12AK8,6.0,TTJets (pow),j3PtAK8,4.0,W+jets,nb,7.0,Z+jets,j1Pt,8


,cut,var,source,yield
864,_qual_trg,njetsAK8,Z+jets,2793174.0
865,_qual_trg,njetsAK8,W+jets,3027143.0
866,_qual_trg,njetsAK8,TTJets (pow),1255211.0
867,_qual_trg,njetsAK8,QCD,155942052.0
868,_qual_trg,njetsAK8,mMed 600,171527.0
869,_qual_trg,njetsAK8,baseline,1002.0
870,_qual_trg,njetsAK8,M-2000_mD-1,1421.0
871,_qual_trg,njetsAK8,M-2000_mD-100,1181.0
872,_qual_trg,njetsAK8,M-2000_r-0p1,725.0
873,_qual_trg,njetsAK8,M-2000_r-0p7,1075.0


_qual_trg


...

,QCD,TT,W+Jets,Z+Jets,All
sig,,,,,
mMed 600,0.549796,0.496249,1.259723,1.491107,0.558184
baseline,1.414591,1.303079,3.604490,4.525219,1.436175
M-2000_mD-1,1.988753,1.830817,5.056530,6.338458,2.019098
M-2000_mD-100,1.118187,1.030113,2.849687,3.577994,1.135247
M-2000_r-0p1,1.894344,1.744988,4.827203,6.060547,1.923242
M-2000_r-0p7,0.868102,0.799868,2.213679,2.780629,0.881349
M-2000_a-low,1.320580,1.216433,3.364363,4.223226,1.340728
M-2000_a-high,1.374372,1.265975,3.501363,4.395156,1.395346


_qual_trg_st_njets


,QCD,TT,W+Jets,Z+Jets,All
sig,,,,,
mMed 600,0.546759,0.481689,1.341885,1.724745,0.555167
baseline,1.422139,1.268363,3.869179,5.428675,1.444015
M-2000_mD-1,1.815593,1.618631,4.931134,6.905177,1.843523
M-2000_mD-100,1.203416,1.073317,3.274154,4.593854,1.221926
M-2000_r-0p1,1.960319,1.748290,5.333349,7.483026,1.990469
M-2000_r-0p7,0.809293,0.721949,2.203695,3.094858,0.821745
M-2000_a-low,1.147619,1.023583,3.122763,4.382117,1.165271
M-2000_a-high,1.348109,1.202317,3.667315,5.144694,1.368849


_qual_trg_st_njetsAK8


,QCD,TT,W+Jets,Z+Jets,All
sig,,,,,
mMed 600,0.470398,0.488062,1.191161,1.479584,0.478266
baseline,1.261199,1.324851,3.449423,4.535677,1.282301
M-2000_mD-1,1.816237,1.906862,4.959200,6.511382,1.846626
M-2000_mD-100,0.990191,1.040234,2.708559,3.561888,1.006756
M-2000_r-0p1,1.721382,1.808208,4.708069,6.190782,1.750178
M-2000_r-0p7,0.736915,0.774282,2.016724,2.653216,0.749247
M-2000_a-low,1.205947,1.266756,3.297754,4.335592,1.226123
M-2000_a-high,1.197421,1.257831,3.274682,4.305532,1.217459
